In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from recovar import (RepresentationLearningSingleAutoencoder, 
                     RepresentationLearningDenoisingSingleAutoencoder, 
                     RepresentationLearningMultipleAutoencoder)
from recovar import ClassifierAutocovariance, ClassifierAugmentedAutoencoder, ClassifierMultipleAutoencoder
from kfold_tester import KFoldTester
from evaluator import Evaluator, CropOffsetFilter, LastEarthquakeFilter
from sklearn.metrics import auc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Should be RepresentationLearningSingleAutoencoder, RepresentationLearningDenoisingSingleAutoencoder or RepresentationLearningMultipleAutoencoder
REPRESENTATION_LEARNING_MODEL_CLASS = RepresentationLearningMultipleAutoencoder
CLASSIFIER_MODEL_CLASS = ClassifierMultipleAutoencoder

In [10]:
# Should be RepresentationLearningSingleAutoencoder, RepresentationLearningDenoisingSingleAutoencoder or RepresentationLearningMultipleAutoencoder
REPRESENTATION_LEARNING_MODEL_CLASS = RepresentationLearningMultipleAutoencoder
CLASSIFIER_MODEL_CLASS = ClassifierMultipleAutoencoder

# Split.
SPLIT = 0

rows = []

In [11]:
def _plot_roc(experiment_prefix, resample_eq_ratio):
    filters = [CropOffsetFilter(), LastEarthquakeFilter()]
    evaluator = Evaluator(exp_name = f"{experiment_prefix}{resample_eq_ratio}", 
                          representation_learning_model_class=REPRESENTATION_LEARNING_MODEL_CLASS, 
                          classifier_model_class = CLASSIFIER_MODEL_CLASS, 
                          train_dataset = "custom", 
                          test_dataset = "custom", 
                          filters = filters, 
                          split = 0,
                          apply_resampling=True,
                          resample_eq_ratio=resample_eq_ratio,
                          report_best_val_score_epoch=True,
                          method_params={})

    roc_vectors = evaluator.get_roc_vectors()
    roc_auc = auc(roc_vectors[0]["fpr"], roc_vectors[0]["tpr"])
    print(roc_auc)

    # Create a DataFrame for Seaborn
    roc_data = pd.DataFrame({
        'False Positive Rate': roc_vectors[0]["fpr"],
        'True Positive Rate': roc_vectors[0]["tpr"]
    })

    plt.figure(figsize=(8, 6))
    sns.lineplot(data=roc_data, x='False Positive Rate', y='True Positive Rate', label='ROC Curve')

    # Plot random guess line
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Seismic Detection Model')
    plt.legend()
    plt.grid(True)
    plt.savefig("tpr-fpr.png")

In [12]:
_plot_roc("exp_resample_eq_ratio", 0.2)

2025-08-27 00:11:39.874607: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-08-27 00:11:41.093505: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:447] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2025-08-27 00:11:41.093628: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:451] Memory usage: 6094848 bytes free, 25447170048 bytes total.
2025-08-27 00:11:41.094558: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:461] Possibly insufficient driver version: 470.256.2
2025-08-27 00:11:41.094604: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_impl.h:1592 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Exception encountered when calling layer 'conv1d' (type Conv1D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} DNN library is not found. [Op:Conv2D] name: 

Call arguments received by layer 'conv1d' (type Conv1D):
  • inputs=tf.Tensor(shape=(256, 3014, 3), dtype=float32)